<a href="https://colab.research.google.com/github/realBagher/AI-for-Medicine-Specialization/blob/main/multi_label_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-label-Text Classification 
An interview task represented by DataColab Team based on highly sparsed datasets of new sections 
<br>
our approach to solve this challenge includes 


1.   Cleaning the Data 
2.   Finding the required attributs as per requested by the Datacolab Team 
3. fine-tuning bert-uncased model on the dataset
4. infering the topics on the test set 



* by MohammadReza Bagherian 

## Importing Dependencies 

In [ ]:
import pandas as pd 
import numpy as np 

import urllib.request
import json


import torch 
import torch.nn as nn 
import shutil 
import sys 

!pip install transformers==4.28.0

## Loading the Data 

In [2]:
to_fill = pd.read_csv('/content/to_fill.csv')
stories = pd.read_csv('/content/stories.csv')

In [ ]:
import json

# Open the JSON file
with open('/content/12387.json') as f:
    data12387 = json.load(f)
print(data12387)


with open('/content/16859.json') as f:
    data16859 = json.load(f)
print(data12387)



with open('/content/18246.json') as f:
    data18246 = json.load(f)

print(data12387)

## Prepocessing 


in this section we extract the **sections of interest** in the text based on the starting and ending words provided in to_fill csv 

In [4]:
def extract_sentences(text, start_phrase, end_phrase):
    """
    Extracts all sentences in a given text that are between the start and end phrases.

    Args:
    text (str): The text to extract sentences from.
    start_phrase (str): The phrase that marks the start of the desired sentences.
    end_phrase (str): The phrase that marks the end of the desired sentences.

    Returns:
    text (String): The sentences that are between the start and end phrases.
    """
   
    start_index = text.find(start_phrase)
    end_index = text.find(end_phrase)
    extraction = text[start_index:(end_index+len(end_phrase))]
    return extraction

In [5]:
def extract_soi(id,start_phrase_,end_phrase_):
  """
    Extracts Section of interest from a text document based on the start and end phrases provided in Dataframe.

    Parameters:
    id (int): The id of the text document to select.
    start_phrase (str): The starting phrase of the sentences to extract.
    end_phrase (str): The ending phrase of the sentences to extract.

    Returns:
    list: A list of sentences that match the criteria defined by the start and end phrases.
    """
  text = None 
  if id == 12387:
    text = data12387.get('text')
  elif id == 16859:
    text = data16859.get('text')
  elif id == 18246:
    text = data18246.get('text')
  else:
    return 'error on ' , id

  
 
  return extract_sentences(text,start_phrase_,end_phrase_)


two rows of the to_fill csv had the last word of the section repeated twice which would cause problems for extraction thus we rectified the problem by editing to_fill dataframe 


In [6]:
to_fill.loc[12,'last_words'] = ' '.join(to_fill.loc[12,'last_words'].split()[:-1])

In [7]:
to_fill.loc[1,'last_words'] = ' '.join(to_fill.loc[1,'last_words'].split()[:-1])

In [8]:
to_fill['body'] = to_fill.apply(lambda row : extract_soi(row[2],row[0],row[1]),axis=1)

using the text extracted in previous section we extract the starting and ending time of the phrase using the word pool

In [9]:
def check_phrase_start_time(id, phrase):
    """
       Check if the given phrase is present in the word pool associated with the given id, 
    and return the start time of the first occurrence of the phrase in the word pool. 
    
    Args:
    - id (int): The id associated with the word pool to search in.
    - phrase (str): The phrase to search for in the word pool.
    
    Returns:
    - If a match is found, the start time of the first occurrence of the phrase in the word pool
      is returned. If no match is found, 'no pattern found' is printed and None is returned.
    - If an invalid id is provided, 'error on ' is printed followed by the invalid id and None is returned.
    
       
    """

  
    phrase = phrase.split()
    word_pool = None
    if id == 12387:
        word_pool = data12387.get('words')
    elif id == 16859:
        word_pool = data16859.get('words')
    elif id == 18246:
        word_pool = data18246.get('words')
    else:
        return 'error on ', id
    
    found_match = False
    for i in range(len(word_pool)):
        j = 0
        if word_pool[i]['text'] == phrase[j]:
            temp = i
            while j < len(phrase) - 1:
                temp += 1
                if word_pool[temp]['text'] == phrase[j+1]:
                    j += 1
                else:
                    break
            if j == len(phrase) - 1:
                found_match = True
                print('pattern found:', i,word_pool[i]['text'],' -at time: ' ,word_pool[i]['start'])
                return word_pool[i]['start']
    if not found_match:
        print('no pattern found')


In [ ]:
to_fill['start'] = to_fill.apply(lambda row : check_phrase_start_time(row[2],row[0]),axis=1)

In [11]:
def check_phrase_end_time(id, phrase):
    """
    Check if the given phrase is present in the word pool associated with the given id,
    and return the end time of the last occurrence of the phrase in the word pool.

    Args:
        id (int): The id associated with the word pool to search in.
        phrase (str): The phrase to search for in the word pool.

    Returns:
        If a match is found, the end time of the last occurrence of the phrase in the word pool
        is returned. If no match is found, 'no pattern found' is printed and None is returned.
        If an invalid id is provided, 'error on' followed by the invalid id is returned as a string.
    """
    phrase = phrase.split()
    word_pool = None
    if id == 12387:
        word_pool = data12387.get('words')
    elif id == 16859:
        word_pool = data16859.get('words')
    elif id == 18246:
        word_pool = data18246.get('words')
    else:
        return 'error on ', id
    
    found_match = False
    for i in range(len(word_pool)):
        j = 0
        if word_pool[i]['text'] == phrase[j]:
            temp = i
            while j < len(phrase) - 1:
                temp += 1
                if word_pool[temp]['text'] == phrase[j+1]:
                    j += 1
                else:
                    break
            if j == len(phrase) - 1:
                found_match = True
                print('pattern found:', i,word_pool[temp]['text'],' -at time: ' ,word_pool[temp]['start'])
                return word_pool[temp]['start']
    if not found_match:
        print('no pattern found')


In [ ]:
to_fill['end'] = to_fill.apply(lambda row : check_phrase_end_time(row[2],row[1]),axis=1)

29 rows of the stories dataframe have empty body column which hinders the traing thus we remove them from training data 

In [13]:
model_data = stories[stories['body'] != ' ']

In [14]:
topic_set = {'dummy',}

### finding the unique topics of the news based on uuid 

In [15]:
def split_(input,topic_sett):
    
    """
    used for finding the unique uuids of the topics 
    Split the input string by comma (',') and add each split element to the topic_sett set.

    Args:
        input (str): The input string to be split.
        topic_sett (set): The set to which the split elements will be added.

    Returns:
        None. The function modifies the topic_sett set in place.

    Example:
        >>> input = "topic1, topic2, topic3"
        >>> topic_sett = set()
        >>> split_(input, topic_sett)
        >>> print(topic_sett)
        {'topic1', 'topic2', 'topic3'}
    """
    temp = input.split(',')
    for i in temp:
     i = i.replace('[','')
     i = i.replace(']','')
     i = i.replace('\'','')
     i = i.replace(' ','')
     topic_sett.add(i)
    

In [16]:
model_data.apply(lambda row : split_(row[1],topic_set),axis=1)

4       None
8       None
11      None
12      None
15      None
        ... 
5176    None
5177    None
5178    None
5179    None
5180    None
Length: 5152, dtype: object

In [17]:
topic_set.remove('dummy')

mappping each unique uuid to a number for ease of use in one-hot-encoding 

In [18]:
topic_dic = {}
# Example list of values
values = topic_set
# Create a dictionary to map values to numbers
value_to_number = {}

# Assign a unique number to each value and store it in the dictionary
for i, value in enumerate(values):
    value_to_number[value] = i

# Print the resulting dictionary
print(value_to_number)
topic_dic = value_to_number

{'b49207eb-96eb-4b73-b534-adc0ef85022a': 0, 'a58b4b70-1b59-4240-917d-a2165a0ce2f0': 1, 'ebf2991e-4b7d-44c6-927b-a261a7b21d2c': 2, '74e2fab8-689f-4e17-9a1c-e1f92e084f55': 3, '0d817400-3f5d-41e0-929c-c31fdbe75d31': 4, '39822b5f-e37e-43e8-b997-7142fe55c3ea': 5, '9a06646a-e1df-4fca-888e-69658420556b': 6, 'ca197b81-ca86-4792-8c25-2ba7cd4195b5': 7, 'e7cbe38d-c987-4113-aa94-fd77eda451d5': 8, '83a09c6b-5f2f-421f-ae50-b38acca7e008': 9, 'aa1edc37-1a01-414a-bcf7-8517e7c7053d': 10, '96326734-fd82-4350-b45c-513e7eb9147c': 11, '9ff54ded-904b-4e0c-85ce-a3617f5cb913': 12, '6fbf954a-03f9-4782-a65f-783271c9c447': 13, 'f5cdd7f2-9d4d-4ba5-9925-00c1701e30fa': 14}


In [19]:
df = model_data

### one-hot-encoding of the topics 

In [20]:
def encode_labels(indexx, topic, df, _dict):
    """
    Encodes labels for a given index in a DataFrame based on a topic string and a mapping dictionary into ONE-HOT-ENCODING.

    Args:
        indexx (int): Index of the row in the DataFrame to encode labels for.
        topic (str): Topic string containing one or more labels separated by commas.
        df (pandas.DataFrame): DataFrame where the labels will be encoded.
        _dict (dict): Mapping dictionary that maps labels to column names in the DataFrame.

    Returns:
        None: The function modifies the DataFrame in-place by encoding the labels.

    """
    temp = topic.split(',')
    for i in temp:
     i = i.replace('[','')
     i = i.replace(']','')
     i = i.replace('\'','')
     i = i.replace(' ','')
     df.loc[indexx,topic_dic[i]] = 1
    
    

In [21]:
df.apply(lambda row : encode_labels(row.name,row[1],df,topic_dic),axis=1)

4       None
8       None
11      None
12      None
15      None
        ... 
5176    None
5177    None
5178    None
5179    None
5180    None
Length: 5152, dtype: object

In [22]:
df = df.fillna(0)
df.to_csv('df_processed_no_token.csv', index=False)

assuming that the dataset has uniform distribution we use ```df.sample``` to create train and validation datasets for model 

In [23]:
train_size = 0.8
train_df = df.sample(frac=train_size, random_state=200)
val_df = df.drop(train_df.index).reset_index(drop=True)

In [24]:
train_df.drop(['topic'],inplace=True,axis=1)
val_df.drop(['topic'],inplace=True,axis=1)


In [25]:
train_df = train_df.reset_index(drop=True)

In [26]:
len(df),len(train_df),len(train_df)

(5152, 4122, 4122)

In [27]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


## Fine-tuning model

In [28]:
import torch
import pandas as pd
import ast
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments




class MultiLabelDataset(torch.utils.data.Dataset):
    
    """
    A custom PyTorch Dataset for handling multi-label classification tasks.

    Args:
        dataframe (pandas.DataFrame): The input DataFrame containing text and label data.
        tokenizer (transformers.PreTrainedTokenizer): The tokenizer used to tokenize the text data.

    Attributes:
        tokenizer (transformers.PreTrainedTokenizer): The tokenizer used to tokenize the text data.
        data (pandas.DataFrame): The input DataFrame containing text and label data.

    """
    
    def __init__(self, dataframe, tokenizer):
        self.tokenizer = tokenizer
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = self.data.loc[index, 'body']
        labels = self.data.loc[index, [0, 1,2,3,4,5,6,7,8,9,10,11,12,13,14]].values
        labels = labels.astype(float) 
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=512, return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'][0], 
            'attention_mask': encoding['attention_mask'][0], 
            'token_type_ids': encoding['token_type_ids'][0], 
            'labels': torch.tensor(labels)
        }

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=15)



train_dataset = MultiLabelDataset(train_df, tokenizer)
test_dataset = MultiLabelDataset(val_df, tokenizer)



training_args = TrainingArguments(
    output_dir='./bert-finetuned-custom-topic-data',
    evaluation_strategy='epoch',
    save_total_limit=5,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=50,

    
)

def compute_metrics(pred):
    labels = pred.label_ids
    predictions = pred.predictions > 0.5
    f1_micro = f1_score(labels, predictions, average='micro')
    roc_auc_micro = roc_auc_score(labels, pred.predictions, average='micro')
    accuracy = accuracy_score(labels, predictions)
    return {
        'f1_micro': f1_micro,
        'roc_auc_micro': roc_auc_micro,
        'accuracy': accuracy,
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,

)

trainer.train()



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch,Training Loss,Validation Loss,F1 Micro,Roc Auc Micro,Accuracy
1,0.156300,0.147651,0.634084,0.950785,0.378641
2,0.124200,0.129834,0.717988,0.961108,0.461165
3,0.095600,0.125840,0.724837,0.964170,0.478641


TrainOutput(global_step=3093, training_loss=0.15059256977607932, metrics={'train_runtime': 1432.3046, 'train_samples_per_second': 8.634, 'train_steps_per_second': 2.159, 'total_flos': 3254011080136704.0, 'train_loss': 0.15059256977607932, 'epoch': 3.0})

In [29]:
torch.cuda.memory_summary(device=None, abbreviated=False)


'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   1274 MiB |   3850 MiB |  37364 GiB |  37362 GiB |\n|       from large pool |   1273 MiB |   3848 MiB |  37078 GiB |  37077 GiB |\n|       from small pool |      1 MiB |     21 MiB |    285 GiB |    285 GiB |\n|---------------------------------------------------------------------------|\n| Active memory         |   1274 MiB |   3850 MiB |  37364 GiB |  37362 GiB |\n|       from large pool |   1273 MiB |   3848 MiB |

## Inference using the fine-tuned model 

In [30]:
text_samples = to_fill['body'].to_list()

In [31]:
def predict_topics(text_inputs, tokenizer, model):
    """
    Predicts the topics for the given text inputs using a trained multi-label classification model.

    Args:
        text_inputs (str or List[str]): The text inputs to predict topics for.
        tokenizer (transformers.PreTrainedTokenizer): The tokenizer used to tokenize the text inputs.
        model (transformers.PreTrainedModel): The trained model for multi-label classification.

    Returns:
        List[List[str]]: A list of predicted topics for each text input.

    """
    
    device = next(model.parameters()).device  # Get the device of the model

    # Tokenize the text inputs
    encoded_inputs = tokenizer(
        text_inputs,
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors='pt'
    )

    # Move the encoded inputs to the appropriate device
    encoded_inputs = {key: value.to(device) for key, value in encoded_inputs.items()}

    # Make predictions
    with torch.no_grad():
        logits = model(**encoded_inputs).logits

    # Apply sigmoid activation to get probabilities
    probabilities = torch.sigmoid(logits)

    # Move the probabilities and predictions to CPU
    probabilities = probabilities.cpu()

    # Convert probabilities to binary predictions (0 or 1) using a threshold of 0.5
    predictions = (probabilities > 0.5).long()

    # Convert predictions to a list of topics
    topic_labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8',
                    '9', '10', '11', '12', '13', '14']
    predicted_topics = []
    for i in range(len(predictions)):
        topic_indices = [j for j, label in enumerate(predictions[i]) if label == 1]
        topics = [topic_labels[index] for index in topic_indices]
        predicted_topics.append(topics)

    return predicted_topics

# Sanity check 
text_input = text_samples[0]
predicted_topics = predict_topics([text_input], tokenizer, model)
print(predicted_topics)


[['12']]


changing back the one hot encoding into uuids 

In [32]:

label_to_topic_uuid= {value: key for key, value in topic_dic.items()}
label_to_topic_uuid

{0: 'b49207eb-96eb-4b73-b534-adc0ef85022a',
 1: 'a58b4b70-1b59-4240-917d-a2165a0ce2f0',
 2: 'ebf2991e-4b7d-44c6-927b-a261a7b21d2c',
 3: '74e2fab8-689f-4e17-9a1c-e1f92e084f55',
 4: '0d817400-3f5d-41e0-929c-c31fdbe75d31',
 5: '39822b5f-e37e-43e8-b997-7142fe55c3ea',
 6: '9a06646a-e1df-4fca-888e-69658420556b',
 7: 'ca197b81-ca86-4792-8c25-2ba7cd4195b5',
 8: 'e7cbe38d-c987-4113-aa94-fd77eda451d5',
 9: '83a09c6b-5f2f-421f-ae50-b38acca7e008',
 10: 'aa1edc37-1a01-414a-bcf7-8517e7c7053d',
 11: '96326734-fd82-4350-b45c-513e7eb9147c',
 12: '9ff54ded-904b-4e0c-85ce-a3617f5cb913',
 13: '6fbf954a-03f9-4782-a65f-783271c9c447',
 14: 'f5cdd7f2-9d4d-4ba5-9925-00c1701e30fa'}

In [33]:
def encode_predictions(text):
    uuid_output = []
    predicted_topics = predict_topics(text, tokenizer, model)
    for j in predicted_topics[0]:
      # print('topic',j,label_to_topic_uuid[int(j)])
      uuid_output.append(label_to_topic_uuid[int(j)])

    # print(predicted_topics)
    return ', '.join(uuid_output)


In [34]:
to_fill.head()

,first_words,last_words,source_video_id,body,start,end
0,Well knew. This morning police need your help,"gunpoint, beating him and stealing his cell ph...",18246,Well knew. This morning police need your help ...,464928,503712
1,a call. San Francisco firefighters rescued a man,all the way down to the ocean.,12387,a call. San Francisco firefighters rescued a m...,359020,384184
2,"Paul. Meanwhile, the state set a record in","night through conservation, some 4000 conserva...",16859,"Paul. Meanwhile, the state set a record in ene...",60704,99404
3,Emergency crews in Florida continue to search for,"in Florida to more than 850,000 homes.",18246,Emergency crews in Florida continue to search ...,505290,533736
4,But even though the state never ordered rolling,feel since their power got cut out needlessly.,16859,But even though the state never ordered rollin...,100910,282596


In [35]:
to_fill['topic'] = to_fill.apply(lambda row : encode_predictions(row[3]),axis=1)

In [36]:
to_fill.to_csv('/content/filled.csv')